In [ ]:
!pip -q install ultralytics opencv-python

import cv2, os, time
from datetime import datetime
from ultralytics import YOLO

# -------- CONFIG --------
VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/wildboar2.jpg"   # upload video to Colab and set this path
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/best.pt"    # upload your trained model
CONF_TH = 0.70
TARGET_CLASSES = {"monkey", "wild_boar", "giant_squirrel"}  # update to your names
SAVE_DIR = "captures"
FRAME_COUNT = 5
FRAME_INTERVAL_FRAMES = 3     # take 1 frame every 3 frames after motion
COOLDOWN_FRAMES = 40          # ignore triggers for next N frames
MOTION_THRESH = 25
MIN_AREA = 1200

os.makedirs(SAVE_DIR, exist_ok=True)

# -------- MOTION DETECT --------
prev_gray = None
def motion_detect(frame_bgr, thresh=MOTION_THRESH, min_area=MIN_AREA):
    global prev_gray
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    if prev_gray is None:
        prev_gray = gray
        return False
    diff = cv2.absdiff(prev_gray, gray)
    prev_gray = gray
    _, th = cv2.threshold(diff, thresh, 255, cv2.THRESH_BINARY)
    th = cv2.dilate(th, None, iterations=2)
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return any(cv2.contourArea(c) > min_area for c in cnts)

# -------- LOAD MODEL --------
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
cooldown = 0
frame_idx = 0

while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame_idx += 1

    if cooldown > 0:
        cooldown -= 1
        continue

    # 1) motion gate
    if not motion_detect(frame):
        continue

    # 2) extract frames after motion
    frames = [frame]
    for i in range(FRAME_COUNT - 1):
        # skip some frames to simulate time gap
        for _ in range(FRAME_INTERVAL_FRAMES):
            ok, f2 = cap.read()
            if not ok:
                break
            frame_idx += 1
        if not ok:
            break
        frames.append(f2)

    # 3) run YOLO on extracted frames
    best = None  # (name, conf, img)
    for img in frames:
        res = model.predict(source=img, imgsz=640, conf=0.25, verbose=False)
        for r in res:
            if r.boxes is None:
                continue
            for b in r.boxes:
                cls_id = int(b.cls.item())
                conf = float(b.conf.item())
                name = r.names[cls_id]
                if name in TARGET_CLASSES and conf >= CONF_TH:
                    if best is None or conf > best[1]:
                        best = (name, conf, img)

    if best:
        animal, conf, img = best
        ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        out = os.path.join(SAVE_DIR, f"{ts}_{animal}_{conf:.2f}.jpg")
        cv2.imwrite(out, img)
        print(f"[TRIGGER] {animal} {conf:.2f} -> saved {out}")
        cooldown = COOLDOWN_FRAMES

cap.release()
print("Done.")
